Agora vamos tentar identificar o que seria ideal em um deck que possui como alvo a carta `jirachi`

In [1]:
POKEMON_ALVO = 'Jirachi'
CONSIDERAR_TREINADORES = True
ANO_ANALISAR = 2019

# quantidade de cartas de 1 tipo que deve exitir no dataset para ser considerado evitando outliers 
CARTAS_MINIMAS = 20

## Bibliotecas utilizadas e carregando dataset

In [2]:
import numpy as np
import pandas as pd
import pysubgroup as ps

df_original = pd.read_csv('./Data/tournaments.csv')
df = df_original.copy()


## Eliminar cartas de treinador se quiser

In [3]:
if not CONSIDERAR_TREINADORES:
    df = df[df['type_card'] != 'Trainer']

## Filtragem de colunas no dataset e limpeza de linhas com coluna regiao do torneio vazias

In [4]:
columns_to_remove = [
    'id_card', 'amount_card', 'price_card', 'combo_type_id', 'name_tournament',
    'combo_type_name', 'name_player', 'category_tournament', 'type_card',
    'month_tournament', 'day_tournament', 'valid_rotation_at_tournament', 
    'rotation_name', 'year_begin', 'all_time_score', 'country_tournament', 'country_player',
    'month_begin', 'day_begin',
]
df.drop(columns=columns_to_remove, inplace=True)
df = df[df['region_tournament'].notna()]

df.head(1)

,name_card,energy_type_card,id_player,ranking_player_tournament,id_tournament,region_tournament,year_tournament
7462,Charmander,Fire,649,1,385,SA,2023


## Separando por regiao para ver oque sera recomendado nos decks para montar um deck com o `POKEMON_ALVO`

In [5]:
def elimina_cards_menos_usados(df):
    cartas = df['name_card'].unique()
    for carta in cartas:
        if len(df[df['name_card'] == carta]) < CARTAS_MINIMAS:
            df = df[df['name_card'] != carta]
    return df

def agrupa_deck(df_region):
    #filtra torneios da regiao
    torneios = df_region['id_tournament'].unique()

    lista_decks = []
    for torneio in torneios:
        df_torneio = df_region[df_region['id_tournament'] == torneio]
        jogadores = df_torneio['id_player'].unique()
        for jogador in jogadores:
            df_jogador = df_torneio[df_torneio['id_player'] == jogador]
            cartas_jogador = df_jogador['name_card'].unique().tolist()
            lista_decks.append(cartas_jogador)
    return lista_decks

def subgrupo_dataset_region(df_region):
    lista_decks = agrupa_deck(df_region)
    cartas = df_region['name_card'].unique()
    data = []
    for deck_rank in lista_decks:
        pokemons_no_deck = {carta: (carta in deck_rank[:len(deck_rank)-1]) for carta in cartas}
        data.append(pokemons_no_deck)
    # Converte a lista de dicionários em um DataFrame
    df_decks_cartas = pd.DataFrame(data)

    return df_decks_cartas

def calcula_subgrupo(df):
    target = ps.BinaryTarget(POKEMON_ALVO, True)
    search_space = ps.create_selectors(df, ignore=[POKEMON_ALVO])
    task = ps.SubgroupDiscoveryTask(
        df,
        target, 
        search_space, 
        result_set_size=20, 
        depth=5, 
        qf=ps.WRAccQF()  # Quality function to evaluate subgroups
    )
    result = ps.BeamSearch().execute(task)
    return result

### Por regiao

In [6]:
regions = df['region_tournament'].unique()
for region in regions:
    print(f"Região: {region}\n")
    df_region = df[(df['region_tournament'] == region) & (df['year_tournament'] == ANO_ANALISAR)]

    # verifica se tem torneios na regiao
    if len(df_region) == 0:
        print(f'Sem torneios na região {region} no ano {ANO_ANALISAR}\n')
        print("-------------------------------------------"*3)
        continue

    # verifica se tem a carta POKEMON_ALVO na regiao
    if POKEMON_ALVO not in df_region['name_card'].unique():
        print(f'Sem a carta {POKEMON_ALVO} na região {region}\n')
        print("-------------------------------------------"*3)
        continue

    df_region = elimina_cards_menos_usados(df_region)
    dataset = subgrupo_dataset_region(df_region)
    result = calcula_subgrupo(dataset)

    # imprime resultados
    for row in result.to_dataframe().itertuples():
        print(f"Subgrupo: {row.subgroup}")
    print()
    print(result.to_dataframe(), "\n")
    print("-------------------------------------------"*3)

Região: SA

Subgrupo: Pokégear 3.0==False AND Switch==True
Subgrupo: Chaotic Swell==False AND Pokégear 3.0==False AND Switch==True
Subgrupo: Custom Catcher==False AND Pokégear 3.0==False AND Switch==True
Subgrupo: Chaotic Swell==False AND Custom Catcher==False AND Pokégear 3.0==False AND Switch==True
Subgrupo: Custom Catcher==False AND Switch==True
Subgrupo: Chaotic Swell==False AND Custom Catcher==False AND Switch==True
Subgrupo: Pokégear 3.0==False AND Reset Stamp==True AND Switch==True
Subgrupo: Great Catcher==True AND Pokégear 3.0==False AND Switch==True
Subgrupo: Chaotic Swell==False AND Pokégear 3.0==False AND Reset Stamp==True AND Switch==True
Subgrupo: Chaotic Swell==False AND Great Catcher==True AND Pokégear 3.0==False AND Switch==True
Subgrupo: Acro Bike==False AND Custom Catcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Chaotic Swell==False AND Custom Catcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Pokégear 3.0==False AND Switch==True
Subgrupo: Ac

### Ignorando Regioes

In [7]:
df_all = df[(df['year_tournament'] == ANO_ANALISAR)]

# verifica se tem torneios
if len(df_all) == 0:
    print(f'Sem torneios no ano {ANO_ANALISAR}')
    print("-------------------------------------------"*3)
    exit()

# verifica se tem a carta POKEMON_ALVO
if POKEMON_ALVO not in df_all['name_card'].unique():
    print(f'Sem a carta {POKEMON_ALVO} no ano {ANO_ANALISAR}')
    print("-------------------------------------------"*3)
    exit()

# calcula subgrupo
df_all = elimina_cards_menos_usados(df_all)
dataset = subgrupo_dataset_region(df_all)
result = calcula_subgrupo(dataset)

# imprime resultados
for row in result.to_dataframe().itertuples():
    print(f"Subgrupo: {row.subgroup}")
print()
print(result.to_dataframe(), "\n")
print("-------------------------------------------"*3)

Subgrupo: Escape Board==True AND Green's Exploration==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Zorua==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Zoroark-GX==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Zoroark-GX==False AND Zorua==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Multi Switch==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Multi Switch==False AND Zorua==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Multi Switch==False AND Zoroark-GX==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Multi Switch==False AND Zoroark-GX==False AND Zorua==False
Subgrupo: Chaotic Swell==False AND Escape Board==True AND Green's Exploration==False
Subgrupo: Chaotic Swell==False AND Escape Board==True AND Green's Exploration==False AND Zorua==False
Subgrupo: Chaotic Swell==False AND Escape Board==True AND Green's Exploration==Fa

## Conclusao

Ao analisar somente cartas do tipo `pokemon`, podemos perceber que as cartas `Tapu Koko ♢` em regioes de `EU` e `Dedenne-GX` na regiao de `SA` seriam cartas com alta sinergia com a carta `Jirachi`, sendo bem relevantes em decks no ano de 2019 que podemos tambem notar no `fpgrowth` quando analisamos cartas recorrentes utilizadas.
Quando analisamos para todos os tipos de cartas tanto `pokemon` quanto `treinador`, vemos que as cartas `Escape Board`, `Switch` e `Guzma`  seriam cartas que estao quase sempre presentes em decks com a carta `Jirachi` ao ponto de ofuscar regras que dizem a cartas `Tapu Koko ♢` e `Dedenne-GX` serem uteis no deck.

De curiosidade foi pesquisado na internet e foi descoberto as seguintes caracteristicas nas 6 cartas muito utilizadas na época:
1. `Jirachi`:
- Habilidade: Stellar Wish: Olhe as cinco cartas do topo do seu baralho, escolha uma carta de Treinador que encontrar ali e coloque-a na sua mão. Depois, Jirachi é colocado no banco e fica adormecido.

2. `Tapu Koko ♢`:
- Habilidade: Dance of the Ancients: Uma vez durante o turno do jogador, se Tapu Koko estiver no banco, pode-se mover duas Energias Elétricas da pilha de descarte para dois Pokémon no banco. Depois, Tapu Koko é removido do jogo.
- Sinergia com Jirachi: Tapu Koko ajuda a energizar rapidamente os Pokémon no banco, permitindo que Jirachi encontre cartas de Treinador essenciais para manter o fluxo do jogo.

3. `Dedenne-GX`:
- Habilidade: Dedechange: Quando o jogador joga Dedenne-GX da mão para o banco, pode descartar a mão atual e comprar seis novas cartas.
- Sinergia com Jirachi: Dedenne-GX ajuda a renovar a mão do jogador, garantindo que ele sempre tenha acesso a cartas de Treinador úteis que Jirachi pode buscar e jogar. Isso aumenta a consistência e a velocidade do baralho.

4. `Escape Board`:
- Efeito: O Pokémon ao qual esta carta está ligada tem seu custo de recuo reduzido em uma Energia Incolor e pode recuar mesmo se estiver Adormecido ou Paralisado.
- Sinergia com Jirachi: Permite que Jirachi, após usar Stellar Wish e ficar Adormecido, recue para o banco sem custo, permitindo a entrada de outro Pokémon Ativo para continuar a estratégia.

5. `Switch`:
- Efeito: Troque o seu Pokémon Ativo por um dos seus Pokémon no banco.
- Sinergia com Jirachi: Permite que Jirachi recue para o banco sem necessidade de energia, mesmo se não houver um Escape Board disponível. Isso mantém a habilidade Stellar Wish ativa em múltiplos turnos, proporcionando uma busca constante de cartas de Treinador.

6. `Guzma`:
- Efeito: Escolha um dos Pokémon no banco do seu oponente e traga-o para a posição de Pokémon Ativo. Em seguida, troque seu Pokémon Ativo por um dos seus Pokémon no banco.
- Sinergia com Jirachi: Além de permitir a manipulação do campo do oponente, Guzma permite que Jirachi recue para o banco e outro Pokémon ative seus ataques ou habilidades. Isso é especialmente útil quando você precisa retirar Jirachi do campo sem gastar energia ou cartas adicionais.

Assim, essas cartas juntas formavam uma combinação muito eficiente:
- Jirachi fornece busca e consistência com sua habilidade Stellar Wish.
- Tapu Koko acelera a energização dos Pokémon no banco.
- Escape Board e Switch garantem que Jirachi possa recuar facilmente após usar sua habilidade.
- Dedenne-GX renova a mão do jogador, garantindo um fluxo constante de cartas de Treinador.
- Guzma permite controle do campo adversário e facilita a troca de Pokémon para manter a pressão ou se defender.

Essa combinação cria um baralho fluido e versátil, capaz de responder a diversas situações e manter um fluxo constante de recursos.